In [1]:
import pandas as pd
import tqdm as notebook_tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
import umap
# import joblib
import numpy as np
os.environ["LOKY_MAX_CPU_COUNT"] = "6"  # Adjust the number accordingly
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


d:\Kaggle\Kaggle-HCT-Survival-Predictions\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_path=Path('../data/raw')
train=pd.read_csv(os.path.join(file_path,'train.csv'),index_col='ID')
test=pd.read_csv(os.path.join(file_path,'test.csv'))
sample=pd.read_csv(os.path.join(file_path,'sample_submission.csv'))
data=pd.read_csv(os.path.join(file_path,'data_dictionary.csv'))

In [3]:
numerical_col=['donor_age','age_at_hct']
target_col=['year_hct','efs_time','efs','race_group']
categorical_col=train.drop(numerical_col+target_col,axis=1).columns

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,HistGradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [ ]:
# # features
# cat_col=joblib.load('../feature_store/categorical_col.pkl')
# num_col=joblib.load('../feature_store/numerical_col.pkl')
# target_col=joblib.load('../feature_store/target_col.pkl')

# # features encoding
# one_hot_encoding=joblib.load('../feature_store/one_hot_encoder.pkl')
# ss_encoding=joblib.load('../feature_store/standard_scaler.pkl')

# # dimensionality reduction
# pca_reduction=joblib.load('../feature_store/pca.pkl')
# tsne_reduction=joblib.load('../feature_store/tsne.pkl')
# umap_reduction=joblib.load('../feature_store/umap_encoder.pkl')


In [5]:
encoding=OneHotEncoder()
df_cat=pd.DataFrame(encoding.fit_transform(train[categorical_col]).toarray(),columns=encoding.get_feature_names_out())

ss=StandardScaler()
df_num=pd.DataFrame(ss.fit_transform(train[numerical_col]),columns=ss.get_feature_names_out())

pca=PCA(n_components=2,random_state=42)
df_pca=pd.DataFrame(pca.fit_transform(df_cat),columns=[f'pca_{i}' for i in range(2)])

tsne = TSNE(n_components=2, perplexity=30, random_state=42)
df_tsne=pd.DataFrame(tsne.fit_transform(df_cat),columns=[f'tsne_{i}' for i in range(2)])

umap_reducer=umap.UMAP(n_components=2)
df_umap=pd.DataFrame(umap_reducer.fit_transform(df_cat),columns=[f'umap_{i}' for i in range(2)])

In [11]:
df=pd.concat([df_cat,df_num,train[target_col]],axis=1)

In [12]:
df.head(2)

,dri_score_High,dri_score_High - TED AML case <missing cytogenetics,dri_score_Intermediate,dri_score_Intermediate - TED AML case <missing cytogenetics,dri_score_Low,dri_score_Missing disease status,dri_score_N/A - disease not classifiable,dri_score_N/A - non-malignant indication,dri_score_N/A - pediatric,dri_score_TBD cytogenetics,...,hla_low_res_10_8.0,hla_low_res_10_9.0,hla_low_res_10_10.0,hla_low_res_10_nan,donor_age,age_at_hct,year_hct,efs_time,efs,race_group
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,NaN,-1.358153,2016,42.356,0.0,More than one race
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.952535,0.238416,2008,4.672,1.0,Asian


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,classification_report,confusion_matrix

In [14]:
X=df.drop(['efs_time','efs','race_group'],axis=1)
y=df.efs
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.8,random_state=42,stratify=y)

In [15]:
model=CatBoostClassifier(random_state=42,verbose=False)
model.fit(X_train,y_train)
confusion_matrix(y_valid,model.predict(X_valid))

array([[1623, 1031],
       [ 655, 2451]], dtype=int64)

In [16]:
model=RandomForestClassifier(random_state=42)
model.fit(X_train.fillna(-999),y_train)
confusion_matrix(y_valid,model.predict(X_valid.fillna(-999)))

array([[1468, 1186],
       [ 681, 2425]], dtype=int64)

In [17]:
model=HistGradientBoostingClassifier()
model.fit(X_train,y_train)
confusion_matrix(y_valid,model.predict(X_valid))

array([[1581, 1073],
       [ 647, 2459]], dtype=int64)

In [18]:
model=XGBClassifier(random_state=42)
X_train.columns=[f'feature_{i}' for i in range(X_train.shape[1])]
X_valid.columns=[f'feature_{i}' for i in range(X_train.shape[1])]
model.fit(X_train,y_train)
confusion_matrix(y_valid,model.predict(X_valid))

array([[1595, 1059],
       [ 774, 2332]], dtype=int64)

In [ ]:
model=LGBMClassifier(random_state=42)
# X_train.columns=[f'feature_{i}' for i in range(X_train.shape[1])]
# X_valid.columns=[f'feature_{i}' for i in range(X_train.shape[1])]
model.fit(X_train,y_train)
confusion_matrix(y_valid,model.predict(X_valid))

[LightGBM] [Info] Number of positive: 12426, number of negative: 10614
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005656 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1092
[LightGBM] [Info] Number of data points in the train set: 23040, number of used features: 287
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.539323 -> initscore=0.157617
[LightGBM] [Info] Start training from score 0.157617


array([[1585, 1069],
       [ 651, 2455]], dtype=int64)

: 

In [41]:
model=LGBMClassifier(random_state=42)
# X_train.columns=[f'feature_{i}' for i in range(X_train.shape[1])]
# X_valid.columns=[f'feature_{i}' for i in range(X_train.shape[1])]
model.fit(X_train,y_train)
confusion_matrix(y_valid,model.predict(X_valid))

array([[1585, 1069],
       [ 651, 2455]], dtype=int64)

In [43]:
df=pd.concat([df_tsne,df_num,train[target_col]],axis=1)
X=df.drop(['efs_time','efs','race_group'],axis=1)
y=df.efs
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.8,random_state=42,stratify=y)
model=CatBoostClassifier(random_state=42,verbose=False)
model.fit(X_train,y_train)
confusion_matrix(y_valid,model.predict(X_valid))

array([[1375, 1279],
       [ 699, 2407]], dtype=int64)

In [44]:
df=pd.concat([df_pca,df_num,train[target_col]],axis=1)
X=df.drop(['efs_time','efs','race_group'],axis=1)
y=df.efs
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.8,random_state=42,stratify=y)
model=CatBoostClassifier(random_state=42,verbose=False)
model.fit(X_train,y_train)
confusion_matrix(y_valid,model.predict(X_valid))

array([[1255, 1399],
       [ 688, 2418]], dtype=int64)

In [45]:
df=pd.concat([df_umap,df_num,train[target_col]],axis=1)
X=df.drop(['efs_time','efs','race_group'],axis=1)
y=df.efs
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.8,random_state=42,stratify=y)
model=CatBoostClassifier(random_state=42,verbose=False)
model.fit(X_train,y_train)
confusion_matrix(y_valid,model.predict(X_valid))

array([[1339, 1315],
       [ 710, 2396]], dtype=int64)

In [ ]:
df=pd.concat([df_tsne,df_pca,df_umap,df_num,train[target_col]],axis=1)
X=df.drop(['efs_time','efs','race_group'],axis=1)
y=df.efs
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.8,random_state=42,stratify=y)
model=CatBoostClassifier(random_state=42,verbose=False)
model.fit(X_train,y_train)
confusion_matrix(y_valid,model.predict(X_valid))

array([[1387, 1267],
       [ 709, 2397]], dtype=int64)

In [ ]:
df=pd.concat([df_cat,df_tsne,df_pca,df_umap,df_num,train[target_col]],axis=1)
X=df.drop(['efs_time','efs','race_group'],axis=1)
y=df.efs
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.8,random_state=42,stratify=y)
model=CatBoostClassifier(random_state=42,verbose=False)
model.fit(X_train,y_train)
confusion_matrix(y_valid,model.predict(X_valid))

array([[1625, 1029],
       [ 658, 2448]], dtype=int64)

In [48]:
df=pd.concat([df_cat,df_tsne,df_umap,df_num,train[target_col]],axis=1)
X=df.drop(['efs_time','efs','race_group'],axis=1)
y=df.efs
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.8,random_state=42,stratify=y)
model=CatBoostClassifier(random_state=42,verbose=False)
model.fit(X_train,y_train)
confusion_matrix(y_valid,model.predict(X_valid))

array([[1642, 1012],
       [ 645, 2461]], dtype=int64)

In [50]:
df=pd.concat([df_cat,df_tsne,df_num,train[target_col]],axis=1)
X=df.drop(['efs_time','efs','race_group'],axis=1)
y=df.efs
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.8,random_state=42,stratify=y)
model=CatBoostClassifier(random_state=42,verbose=False)
model.fit(X_train,y_train)
confusion_matrix(y_valid,model.predict(X_valid))

array([[1631, 1023],
       [ 656, 2450]], dtype=int64)

In [7]:
df=pd.concat([df_cat,df_tsne,df_umap,df_num,train[target_col]],axis=1)
X=df.drop(['efs_time','efs','race_group'],axis=1)
y=df.efs
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.8,random_state=42,stratify=y)
model=CatBoostClassifier(random_state=42,verbose=False)
model.fit(X_train,y_train)
confusion_matrix(y_valid,model.predict(X_valid))

array([[1618, 1036],
       [ 664, 2442]], dtype=int64)

In [11]:
df=pd.concat([df_cat,df_tsne,df_num,train[target_col]],axis=1)
X=df.drop(['efs_time','efs','race_group'],axis=1)
y=df.efs
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.8,random_state=42,stratify=y)
model=CatBoostClassifier(random_state=42,verbose=False)
model.fit(X_train,y_train)
confusion_matrix(y_valid,model.predict(X_valid))

array([[1631, 1023],
       [ 656, 2450]], dtype=int64)

In [26]:
df=pd.concat([df_cat,df_umap,df_pca,df_num,train[target_col]],axis=1)
X=df.drop(['efs_time','efs','race_group'],axis=1)
y=df.efs
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.8,random_state=42,stratify=y)
model=CatBoostClassifier(random_state=42,verbose=False)
model.fit(X_train,y_train)
confusion_matrix(y_valid,model.predict(X_valid))

array([[1621, 1033],
       [ 669, 2437]], dtype=int64)

In [27]:
print(classification_report(y_valid,model.predict(X_valid)))

              precision    recall  f1-score   support

         0.0       0.71      0.61      0.66      2654
         1.0       0.70      0.78      0.74      3106

    accuracy                           0.70      5760
   macro avg       0.71      0.70      0.70      5760
weighted avg       0.70      0.70      0.70      5760



In [93]:
df_temp=train.loc[y_train.index,['efs','efs_time']].copy()
# df_temp.loc[(df_temp.efs==1)&(df_temp.efs_time>10),'efs']=0


In [95]:
df_temp.loc[(df_temp.efs==1)&(df_temp.efs_time>10),'efs'].index

Int64Index([ 2655, 20984,   262, 10756, 18645, 10645, 16998,  8555, 19415,
            22188,
            ...
            20849,  7764,  5145, 27968, 24841, 19904, 26993, 11668, 18897,
            13374],
           dtype='int64', length=901)

In [89]:
# df_temp[df_temp.efs==0].describe()

In [90]:
# df_temp[df_temp.efs==0].describe()

In [110]:
X_train.loc[(df_temp.loc[((df_temp.efs==1)&(df_temp.efs_time<10))|(df_temp.efs==0),'efs'].index),:]

,dri_score_High,dri_score_High - TED AML case <missing cytogenetics,dri_score_Intermediate,dri_score_Intermediate - TED AML case <missing cytogenetics,dri_score_Low,dri_score_Missing disease status,dri_score_N/A - disease not classifiable,dri_score_N/A - non-malignant indication,dri_score_N/A - pediatric,dri_score_TBD cytogenetics,...,hla_low_res_10_9.0,hla_low_res_10_10.0,hla_low_res_10_nan,umap_0,umap_1,pca_0,pca_1,donor_age,age_at_hct,year_hct
26919,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,-3.184205,9.072546,-2.398211,-0.547498,-1.201915,1.245547,2016
25266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,4.700313,1.120443,2.228583,2.376828,-0.701101,-1.826206,2016
11236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,-3.733342,8.847235,-2.462920,-0.323898,-1.580772,0.867530,2018
18891,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,-4.514463,7.551953,-2.023453,-0.830030,2.028398,-1.826206,2014
18917,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,11.558124,8.428111,3.728572,-1.888998,1.519388,-0.318915,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,-3.305656,9.955767,-2.432485,-0.343952,-0.308999,0.639226,2018
5916,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,2.444310,4.102546,0.323434,1.706999,-0.709362,1.096969,2011
8930,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,2.145144,4.006128,-0.631795,1.308960,1.046505,0.338477,2016
7146,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.134377,5.598875,-1.394535,0.799542,0.507398,1.287301,2018


In [ ]:
df=pd.concat([df_cat,df_umap,df_pca,df_num,train[target_col]],axis=1)
X=df.drop(['efs_time','efs','race_group'],axis=1)
y=df.efs
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.8,random_state=42,stratify=y)
y_train=df_temp.efs
model=CatBoostClassifier(random_state=42,verbose=False)
model.fit(X_train,y_train)
confusion_matrix(y_valid,model.predict(X_valid))

array([[2265,  389],
       [1925, 1181]], dtype=int64)

In [123]:
df=pd.concat([df_cat,df_umap,df_num,train[target_col]],axis=1)
X=df.drop(['efs_time','efs','race_group'],axis=1)
y=df.efs
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.8,random_state=42,stratify=y)
X_train=X_train.loc[(df_temp.loc[((df_temp.efs==1)&(df_temp.efs_time<40))|((df_temp.efs==0)&(df_temp.efs_time<200)),'efs'].index),:]
y_train=y_train.loc[(df_temp.loc[((df_temp.efs==1)&(df_temp.efs_time<40))|((df_temp.efs==0)&(df_temp.efs_time<200)),'efs'].index)]
model=CatBoostClassifier(random_state=42,verbose=False)
model.fit(X_train,y_train)
confusion_matrix(y_valid,model.predict(X_valid))

array([[1615, 1039],
       [ 651, 2455]], dtype=int64)

In [7]:
from sklearn.ensemble import VotingClassifier,StackingClassifier

In [ ]:
df_temp=train.loc[y_train.index,['efs','efs_time']].copy()
df=pd.concat([df_cat,df_umap,df_num,train[target_col]],axis=1)
X=df.drop(['efs_time','efs','race_group'],axis=1)
y=df.efs
X.columns=[f'feature_{i}' for i in range(X.shape[1])]
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.8,stratify=y)
X_train=X_train.loc[(df_temp.loc[((df_temp.efs==1)&(df_temp.efs_time<7))|((df_temp.efs==0)&(df_temp.efs_time<200)),'efs'].index),:]
y_train=y_train.loc[(df_temp.loc[((df_temp.efs==1)&(df_temp.efs_time<7))|((df_temp.efs==0)&(df_temp.efs_time<200)),'efs'].index)]
model=Pipeline(steps=[
    ('voting',VotingClassifier([
    ('cat',CatBoostClassifier(random_state=42,verbose=False)),
    ('xgb',XGBClassifier(random_state=42))
    ]))
])
model.fit(X_train,y_train)
confusion_matrix(y_valid,model.predict(X_valid))

In [ ]:
print(classification_report(y_valid,model.predict(X_valid)))

              precision    recall  f1-score   support

         0.0       0.67      0.68      0.68      2654
         1.0       0.72      0.71      0.72      3106

    accuracy                           0.70      5760
   macro avg       0.70      0.70      0.70      5760
weighted avg       0.70      0.70      0.70      5760



: 

In [112]:
print(classification_report(y_valid,model.predict(X_valid)))

              precision    recall  f1-score   support

         0.0       0.69      0.65      0.67      2654
         1.0       0.71      0.76      0.74      3106

    accuracy                           0.71      5760
   macro avg       0.70      0.70      0.70      5760
weighted avg       0.71      0.71      0.71      5760



In [ ]:
array([[1621, 1033],
       [ 669, 2437]], dtype=int64)

In [39]:
# encoding=OneHotEncoder()
df_test_cat=pd.DataFrame(encoding.transform(test[categorical_col]).toarray(),columns=encoding.get_feature_names_out())

# ss=StandardScaler()
df_test_num=pd.DataFrame(ss.transform(test[numerical_col]),columns=ss.get_feature_names_out())

# pca=PCA(n_components=2,random_state=42)
df_test_pca=pd.DataFrame(pca.transform(df_test_cat),columns=[f'pca_{i}' for i in range(2)])

# tsne = TSNE(n_components=2, perplexity=30, random_state=42)
# df_test_tsne=pd.DataFrame(tsne.transform(df_test_cat),columns=[f'tsne_{i}' for i in range(2)])

# umap_reducer=umap.UMAP(n_components=2)
df_test_umap=pd.DataFrame(umap_reducer.transform(df_test_cat),columns=[f'umap_{i}' for i in range(2)])

In [42]:
df_test=pd.concat([df_test_cat,df_test_umap,df_test_pca,df_test_num,test['year_hct']],axis=1)

In [51]:
model.predict_proba(df_test)[:,1]

array([0.1645842 , 0.70195895, 0.05603089])

In [54]:
test_sub=test[['ID']].copy()
test_sub.loc[:,'prediction']=model.predict_proba(df_test)[:,1]
test_sub

,ID,prediction
0,28800,0.164584
1,28801,0.701959
2,28802,0.056031


In [ ]:
# from pyffm import PyFFM

        _________  _.
.__    |__ |__ | \/ | 
|__\__/|   |   |    |
|   /



In [8]:
training_params = {'epochs': 2, 'reg_lambda': 0.002}
pyffm = PyFFM(model='ffm', training_params=training_params)

In [9]:
df=pd.concat([df_cat,df_tsne,df_num,train[target_col]],axis=1)
X=df.drop(['efs_time','race_group'],axis=1)
X.rename(columns={'efs':'click'},inplace=True)
X_train,X_valid=train_test_split(X,train_size=0.6,random_state=42)

In [10]:
pyffm.train(X_train)

[2025-03-03 10:36:55,598.598] -     INFO: pyffm.pyffm -- Formatting dataframe
[2025-03-03 10:37:28,491.491] -     INFO: pyffm.engine.ffm_engine -- Creating ffm model with 301 fields and 62910 features.
[2025-03-03 10:37:29,774.774] -     INFO: pyffm.engine.ffm_engine -- Epoch 0
[2025-03-03 10:37:29,774.774] -     INFO: pyffm.engine.ffm_engine -- Training on 15552 rows.


: 

In [14]:
pred=pyffm.predict(X_valid)

[2025-03-03 09:58:18,440.440] -     INFO: pyffm.pyffm -- Formatting predict data
[2025-03-03 09:58:18,440.440] -     INFO: pyffm.pyffm -- Formatting dataframe
[2025-03-03 09:59:28,595.595] -     INFO: pyffm.engine.ffm_engine -- Batch predicting...


In [16]:
confusion_matrix(X_valid.click,pred)

array([[10621,     0],
       [12419,     0]], dtype=int64)

In [17]:
X_valid.click.value_counts()

1.0    12419
0.0    10621
Name: click, dtype: int64

In [18]:
X_train.click.value_counts()

1.0    3113
0.0    2647
Name: click, dtype: int64

In [ ]:
array([[1623, 1031],
       [ 655, 2451]], dtype=int64)

In [ ]:
# import ffm

# # Prepare data in (field, index, value) format
# X = [[(1, 2, 1), (2, 3, 1), (3, 5, 1)],
#      [(1, 0, 1), (2, 3, 1), (3, 7, 1)],
#      [(1, 1, 1), (2, 3, 1), (3, 7, 1), (3, 9, 1)],
#      [(1, 0, 1), (2, 3, 1), (3, 5, 1)]]
# y = [1, 1, 0, 1]

# ffm_data = ffm.FFMData(X, y)

# # Train model
# model = ffm.FFM(eta=0.1, lam=0.0001, k=4)
# model.fit(ffm_data, num_iter=10)

# # Make predictions
# print(model.predict_proba(ffm_data))
